**Scenarios in the code:** <br>

Payment ACCEPTED <br>
    1. Payment amount is correct and Invoice Number, Client ID or the number in Invoice Name is included in the Payment 

Payment TO BE CHECKED <br>
    2. Payment amount is incorrect and Invoice Number, Client ID or the number in Invoice Name is included in the Payment

Payment ACCEPTED <br>
    3. Payment amount is correct and Name and a Surname on the incoming Payment and the Invoice list are the same 

Payment ACCEPTED <br>
    4. Payment amount is correct and there is similarity of the Adress in the Payment Name and the Property Adress

Payment TO BE CHECKED <br>
    5. Payment amount is incorrect and there is similarity of the Adress in the Payment Name and the Property Adress

Payment TO BE CHECKED <br>
    6. Only payment amount is the same as one unassigned in the database

Payment TO BE CHECKED <br>
    7. Not any information from the payments, matches with any other information in the database <br>



In [1]:
import pandas as pd
import re
import jellyfish
from decimal import Decimal

# Load payment dataset
payments = pd.read_csv('Datasets/Bank_Payments.csv', delimiter=';', encoding="utf-8")
payments['Payment Date'] = pd.to_datetime(payments['Payment Date'])
# Apply the string replacements
payments['Payed Amount'] = payments['Payed Amount'].str.replace(',', '.')
# Convert the column to Decimal
payments['Payed Amount'] = payments['Payed Amount'].apply(lambda x: Decimal(x))
payments['Transaction Number'] = payments['Transaction Number'].astype(int)

# Load invoice dataset
invoices = pd.read_csv('Datasets/Property_DB_Invoices.csv', delimiter=';', encoding="utf-8")
invoices['Invoice Number'] = invoices['Invoice Number'].astype(int)
invoices['Client ID'] = invoices['Client ID'].astype(int)
invoices['Invoice Date'] = pd.to_datetime(invoices['Invoice Date'])
invoices['Sale Date'] = pd.to_datetime(invoices['Sale Date'])
# Apply the string replacements
invoices['Payment Amount'] = invoices['Payment Amount'].str.replace(',', '.')

# Convert the column to Decimal
invoices['Payment Amount'] = invoices['Payment Amount'].apply(lambda x: Decimal(x))

In [2]:
# To be deleted
payments = payments.sample(frac=1, random_state=42).reset_index(drop=True)

In [3]:
# Initialize match status columns
payments['Merged'] = "Not Merged"
invoices['Transaction Number'] = 0
invoices['Matched With'] = "Not Matched"
invoices['Status'] = "Not Matched"



In [4]:
# Define helper function to split text
def split(txt, seps):
    
    """
    Splits a given text string based on multiple separators, extracts date patterns, and converts numeric substrings to integers.
    
    Args:
        txt (str): The input text string to split.
        seps (list): A list of separator characters to use for splitting the input text.
    
    Returns:
        list: A list containing the split text elements, with numeric elements converted to integers and date patterns as integers.
    
    Example:
        >>> split("Invoice 123, 10/11/2022", [',', ';', ' ', '_'])
        ['Invoice', 123, 10112022]
    """
    
    default_sep = seps[0]

    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)

    pattern = r"\d{2}/\d{2}/\d{4}"
    matches = re.findall(pattern, txt)
    txt = re.sub(pattern, "", txt)

    lst = [i.strip() for i in txt.split(default_sep)]

    for i in range(len(lst)):
        if lst[i].isdigit():
            lst[i] = int(lst[i])
        elif lst[i].replace(".", "", 1).isdigit():
            lst[i] = int(float(lst[i]))

    for match in matches:
        match_int = int(match.replace("/", ""))
        lst.append(match_int)

    return lst



In [5]:
# definition of string similarity finction
def get_similarity(string1, string2):
    
    """
    Calculates the similarity between two strings using the Levenshtein distance.
    
    Args:
        string1 (str): The first input string for comparison.
        string2 (str): The second input string for comparison.
    
    Returns:
        float: The similarity score between the input strings, ranging from 0 (completely dissimilar) to 1 (identical).
    
    Example:
        >>> get_similarity("hello", "helo")
        0.8
    """
    
    distance = jellyfish.levenshtein_distance(string1, string2)
    similarity = 1 - distance / max(len(string1), len(string2))
    return similarity

In [6]:
# Define separators for payment names
separators = [',', ';', ' ', '_']

In [7]:
# Perform matching operations
for i, payment in payments.iterrows():
    if payment['Merged'] == "Not Merged":
        for j, invoice in invoices.iterrows():
            if invoice["Matched With"] == "Not Matched":
                
                # Spliting strings in Payment Name, str numbers to integers, reformating Invoice Name 10/2023/2023 to int
                payment_name_list = split(payment['Payment Name'], separators)
                
                # Extracting Int number from Invoice Name 10/2023/2023 -> 1020232023 (df2)
                invoice_num_name = re.findall(r'\d+', invoice['Invoice Name'])[:3]
                invoice_num = ''.join(invoice_num_name)
                invoice_num = int(invoice_num)

                # List of any numbers connected to the property 
                nums_to_check = [invoice['Invoice Number'], invoice['Client ID'], invoice_num]

                # Checking if any of the numbers connected to the propery is present in the Payment Name (proper entries)
                for num in nums_to_check:
                    if num in payment_name_list and payment['Payed Amount'] == invoice['Payment Amount']:
                        invoices.at[j, "Matched With"] = payment['Unique Change Code']
                        invoices.at[j, 'Transaction Number'] = payment['Transaction Number']
                        invoices.at[j, 'Status'] = "ACCEPTED"
                        payments.at[i, "Merged"] = "Correct Amount, Payment data ok"
                        break

                
                # Checking if any of the numbers connected to the propery is present 
                # in the Payment Name but payment amount is wrong
                    elif num in payment_name_list and payment['Payed Amount'] != invoice['Payment Amount']:
                        invoices.at[j, "Matched With"] = payment['Unique Change Code']
                        invoices.at[j, 'Transaction Number'] = payment['Transaction Number']
                        invoices.at[j, 'Status'] = "TO BE CHECKED"
                        payments.at[i, "Merged"] = "Wrong Amount, Payment data ok"
                        break



In [8]:
# Checking Name and a Surname on the Incoming Payment and the Invoice list and checking the Amount
for i, payment in payments.iterrows():
    if payment['Merged'] == "Not Merged":
        for j, invoice in invoices.iterrows():
            if invoice["Matched With"] == "Not Matched":

                if payment['Name and Surname'] == invoice['Name and Surname'] and payment['Payed Amount'] == invoice['Payment Amount']:
                    invoices.at[j, "Matched With"] = payment['Unique Change Code']
                    invoices.at[j, 'Transaction Number'] = payment['Transaction Number']
                    invoices.at[j, 'Status'] = "ACCEPTED"
                    payments.at[i, "Merged"] = "Correct Amount, but only Name and Surname Match"
                    break
                

In [9]:
# Levenshtein Distance Threshold
threshold = 0.6
best_match = None
max_similarity = 0

In [10]:
# Checking the similarity in the Payment Name and the Property Adress and the Payed Amount 
for i, payment in payments.iterrows():
    if payment['Merged'] == "Not Merged":
        for j, invoice in invoices.iterrows():
            if invoice['Matched With'] == "Not Matched":
                similarity = get_similarity(payment['Payment Name'], invoice['Property Adress'])

                if similarity > max_similarity and similarity > threshold:
                    max_similarity = similarity
                    best_match = (i, j)

if best_match:
    i, j = best_match
    if payments['Payed Amount'][i] == invoices['Payment Amount'][j]:
        invoices.at[j, "Matched With"] = payments['Unique Change Code'][i]
        invoices.at[j, 'Transaction Number'] = payment['Transaction Number']
        invoices.at[j, 'Status'] = "ACCEPTED"
        payments.at[i, "Merged"] = "Similar Address, Amount Ok"
        
    else:
        invoices.at[j, "Matched With"] = payments['Unique Change Code'][i]
        invoices.at[j, 'Transaction Number'] = payment['Transaction Number']
        invoices.at[j, 'Status'] = "TO BE CHECKED"
        payments.at[i, "Merged"] = "Address Match, but Check VALUE !"


In [11]:
# Checking only proper Amount (1007_PN_NS_Adr)

for i, payment in payments.iterrows():
    if payment['Merged'] == "Not Merged":
        for j, invoice in invoices.iterrows():
            if invoice['Matched With'] == "Not Matched":
                if payment['Payed Amount'] == invoice['Payment Amount']:
                    invoices.at[j, "Matched With"] = payments['Unique Change Code'][i]
                    invoices.at[j, 'Transaction Number'] = payment['Transaction Number']
                    invoices.at[j, 'Status'] = "TO BE CHECKED"
                    payments.at[i, "Merged"] = "Amount Correct, Data Wrong !"
                    break

# Checking the Payments which does not make sense at all (1013_PN_NS_Adr_PA)

                elif payment['Payed Amount'] != invoice['Payment Amount']:
                    invoices.at[j, "Matched With"] = payments['Unique Change Code'][i]
                    invoices.at[j, 'Transaction Number'] = payment['Transaction Number']
                    invoices.at[j, 'Status'] = "TO BE CHECKED"
                    payments.at[i, "Merged"] = "Wrong Transfer"
                    break   

In [12]:
payments.to_csv('Datasets/merged_payments.csv', index=False)
invoices.to_csv('Datasets/matched_invoices.csv', index=False)

In [13]:
payments

,Unique Change Code,Payment Date,Account Number,Payment Name,Name and Surname,Adreess,Payed Amount,Transaction Number,Merged
0,1012_PN_Adr,2022-11-26,DE49123456780989054321,Sophie Thruman,Sophie Thruman,"Rothenstrasse 5, Hamburg",697.86,4145621430,"Correct Amount, but only Name and Surname Match"
1,1011_PN_Adr,2023-01-20,DE93123456780123456789,99/01/2029,Olivia Martinez,"Gansemarkt 45, Hamburg",1923.19,7823419056,"Correct Amount, but only Name and Surname Match"
2,1001_Unchanged,2022-11-22,DE86123456780098765432,41/11/2022_InHouse_GmbH,Hannah Schneider,"Romerstrasse 1/12, Munich",2222.59,3098574162,"Correct Amount, Payment data ok"
3,1010_PN_Adr,2022-12-10,DE19123456780234567890,68/12/2022_InHouse_GmbH 249503,Charlotte Weber,"Isartorplatz 8, Munich",1142.03,3412765980,"Correct Amount, Payment data ok"
4,1007_PN_NS_Adr,2023-01-10,DE49123456780098765432,Payment,Don Johnson,"Theodor-Heuss-Allee 50, Frankfurt",418.24,5678901234,"Amount Correct, Data Wrong !"
5,1003_Adr_PA,2023-01-20,DE32123456780987654321,81/01/2023_InHouse_GmbH,Hannah Koch,"Steindamm 1, Hamburg",1300,4561892730,"Wrong Amount, Payment data ok"
6,1002_PN,2023-01-16,DE44123456780123456789,521909,Emma Johnson,"Kaiserstrasse 26, Frankfurt",694.98,7809234561,"Correct Amount, Payment data ok"
7,1013_PN_NS_Adr_PA,2022-11-26,DE49123456789999054321,Oranges,Nick West,"5tgh Avenue, New York, USA",100080,9990621430,Wrong Transfer
8,1005_PN_Adr,2022-12-28,DE92123456780123456789,Wohnung Payment December,Lena Schulz,"Viktualienmarkt 6, Munich",335.31,9012837645,"Correct Amount, but only Name and Surname Match"
9,1009_Unchanged,2022-10-06,DE66123456780987654321,39/11/2022_InHouse_GmbH,Sophia Monk,"Friedrich-Ebert-Anlage 49, Berlin",2573.19,9876543210,"Correct Amount, Payment data ok"


In [14]:
invoices

,Unique Match Code,Invoice Number,Client ID,Invoice Date,Sale Date,Invoice Name,Name and Surname,Property Adress,Payment Amount,Transaction Number,Matched With,Status
0,1001,447449,221958,2022-04-11,2022-09-12,41/11/2022_InHouse_GmbH,Hannah Schneider,"Romerstrasse 1/12, Munich",2222.59,3098574162,1001_Unchanged,ACCEPTED
1,1002,521909,771155,2023-01-15,2023-01-28,14/01/2023_InHouse_GmbH,Emma Johnson,"Kaiserstrasse 26, Frankfurt",694.98,7809234561,1002_PN,ACCEPTED
2,1003,358795,231932,2023-01-17,2023-01-17,81/01/2023_InHouse_GmbH,Hannah Koch,"Kaiserplatz 8/2, Potsdam",1287.58,4561892730,1003_Adr_PA,TO BE CHECKED
3,1004,586232,465838,2023-01-15,2023-01-23,10/01/2023_InHouse_GmbH,Ava Davis,"Eschersheimer Landstrasse 14, Frankfurt",1053.74,1287365490,1004_PN_Adr,ACCEPTED
4,1005,600186,359178,2022-11-28,2023-10-01,23/11/2022_InHouse_GmbH,Lena Schulz,"Unter den Linden 15, Berlin",335.31,9012837645,1005_PN_Adr,ACCEPTED
5,1007,256730,744167,2023-01-17,2023-01-21,15/01/2023_InHouse_GmbH,Charlotte Rodriguez,"Schillerstr. 27, Berlin",418.24,5678901234,1007_PN_NS_Adr,TO BE CHECKED
6,1008,970910,210268,2022-11-11,2023-01-20,47/11/2022_InHouse_GmbH,Lea Johnson,"Elbchaussee 401, Hamburg",1753.19,4321890657,1008_PN_NS_Adr_PA,TO BE CHECKED
7,1009,484681,832180,2022-06-11,2023-01-29,39/11/2022_InHouse_GmbH,Sophia Monk,"Friedrich-Ebert-Anlage 49, Berlin",2573.19,9876543210,1009_Unchanged,ACCEPTED
8,1010,249503,154886,2022-12-19,2023-05-01,68/12/2022_InHouse_GmbH,Charlotte Weber,"Bismarckstrasse 5, Hamburg",1142.03,3412765980,1010_PN_Adr,ACCEPTED
9,1011,754811,237337,2023-01-25,2023-01-27,99/01/2023_InHouse_GmbH,Olivia Martinez,"Potsdamer Str. 4, Berlin",1923.19,7823419056,1011_PN_Adr,ACCEPTED
